# Analyze Results of a Hyperparameter Tuning job

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.

---
## Set up the environment
To start the analysis, you must pick the name of the hyperparameter tuning job.

In [1]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')

tuning_job_name = 'xgboost-190425-1638'

## Track hyperparameter tuning job progress
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [2]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

30 training jobs have completed


In [3]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2019, 4, 25, 18, 3, 27, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.9998829960823059},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2019, 4, 25, 18, 15, 11, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:023375022819:training-job/xgboost-190425-1638-027-a3be08f1',
 'TrainingJobName': 'xgboost-190425-1638-027-a3be08f1',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2019, 4, 25, 18, 6, 23, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '1.6091125273891147',
                          'eta': '0.9840675379604573',
                          'max_depth': '9',
                          'min_child_weight': '1.0822314603906955'}}


## Fetch all results as DataFrame
We can list hyperparameters and objective metrics of all training jobs and pick up the training job with the best objective metric.

In [4]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

Number of training jobs with valid objective: 30
{'lowest': 0.4992009997367859, 'highest': 0.9998829960823059}


,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,alpha,eta,max_depth,min_child_weight
3,0.999883,528.0,2019-04-25 18:15:11+00:00,xgboost-190425-1638-027-a3be08f1,Completed,2019-04-25 18:06:23+00:00,1.609113,0.984068,9.0,1.082231
4,0.999852,522.0,2019-04-25 18:11:14+00:00,xgboost-190425-1638-026-e334035a,Completed,2019-04-25 18:02:32+00:00,0.000002,0.275003,10.0,1.175471
7,0.999851,425.0,2019-04-25 17:58:21+00:00,xgboost-190425-1638-023-e35896af,Completed,2019-04-25 17:51:16+00:00,0.000000,0.442461,7.0,1.259111
12,0.999844,438.0,2019-04-25 17:41:28+00:00,xgboost-190425-1638-018-9159e005,Completed,2019-04-25 17:34:10+00:00,1.582257,0.714292,7.0,2.165483
23,0.999844,513.0,2019-04-25 17:06:04+00:00,xgboost-190425-1638-007-373471cc,Completed,2019-04-25 16:57:31+00:00,0.143392,0.857145,9.0,4.840753
29,0.999820,411.0,2019-04-25 16:48:37+00:00,xgboost-190425-1638-001-3f62d0fb,Completed,2019-04-25 16:41:46+00:00,1.380677,0.861549,6.0,3.232443
22,0.999820,497.0,2019-04-25 17:07:30+00:00,xgboost-190425-1638-008-1ed44c79,Completed,2019-04-25 16:59:13+00:00,1.954755,0.881864,9.0,7.757074
6,0.999818,521.0,2019-04-25 18:03:05+00:00,xgboost-190425-1638-024-e5f0a8b3,Completed,2019-04-25 17:54:24+00:00,0.000000,0.274193,10.0,1.189954
28,0.999817,549.0,2019-04-25 16:50:54+00:00,xgboost-190425-1638-002-33d0509e,Completed,2019-04-25 16:41:45+00:00,1.232956,0.898040,10.0,2.630019
15,0.999815,529.0,2019-04-25 17:31:20+00:00,xgboost-190425-1638-015-fc4334d6,Completed,2019-04-25 17:22:31+00:00,0.434530,0.520922,9.0,1.085253


## See TuningJob results vs time
Next we will show how the objective metric changes over time, as the tuning job progresses.  For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance _exploration_ of new areas of parameter space against _exploitation_ of known good areas.  This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [5]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

Loading BokehJS ...

## Analyze the correlation between objective metric and individual hyperparameters 
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For exmaple, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [6]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))